In [1]:
import tensorflow as tf
import numpy as np
import random

In [2]:
def variables(shape):
    σ2, bias = 0.1, 0.1
    U0, U1, U2 = (
        tf.Variable(tf.truncated_normal(shape, stddev=σ2)),
        tf.Variable(tf.truncated_normal(shape, stddev=σ2)),
        tf.Variable(tf.truncated_normal(shape, stddev=σ2)))
    b0, b1, b2 = (
        tf.Variable(tf.constant(bias, shape=[shape[3]])),
        tf.Variable(tf.constant(bias, shape=[shape[3]])),
        tf.Variable(tf.constant(bias, shape=[shape[3]])))
    return (U0, U1, U2), (b0, b1, b2)

In [3]:
def convolutional_gated_recurrent_unit(s, U, b):
    STRIDES = [1, 1, 1, 1]
    u = tf.sigmoid(tf.nn.conv2d(s, U[0], strides=STRIDES, padding='SAME'))
    r = tf.sigmoid(tf.nn.conv2d(s, U[1], strides=STRIDES, padding='SAME'))
    a = tf.tanh(tf.nn.conv2d(r * s, U[2], strides=STRIDES, padding='SAME'))
    return u * s + (1. - u) * a

In [4]:
def vocabulary(n, d):
    σ2 = 0.1
    return tf.Variable(tf.truncated_normal([n, d], stddev=σ2))

In [21]:
def lookup_symbols(sequence, ids):
    return list(map(lambda s: ids.get(s), sequence))

In [6]:
def training_example():
    a, b = random.randint(0, 1024), random.randint(0, 1024)
    c = a + b
    return '%s+%s' % (bin(a)[2:], bin(b)[2:]), bin(c)[2:]

In [7]:
session = tf.InteractiveSession()

In [8]:
SYMBOLS = {
    '0': 0,
    '1': 1,
    '+': 2,
}
DIMENSION = 10
N = 4
L = 4
SHAPE = (3, 3, DIMENSION, DIMENSION)

In [9]:
V = vocabulary(len(SYMBOLS), DIMENSION)
Θ = (variables(SHAPE), variables(SHAPE), variables(SHAPE), variables(SHAPE))

In [10]:
x, y_ = tf.placeholder(tf.int32, shape=[None]), tf.placeholder(tf.int32, shape=[None])

In [11]:
s = tf.expand_dims(tf.pad(tf.expand_dims(tf.nn.embedding_lookup(V, x), -1), tf.constant([[0, 0], [0, 0], [0, 4]])), 0)
s = tf.transpose(s, perm=[0, 1, 3, 2])

In [13]:
for n in range(N):
    for l in range(L):
        s = convolutional_gated_recurrent_unit(s, Θ[l][0], Θ[l][1])

In [22]:
a, b = training_example()
print(a, b, session.run(s, feed_dict={
            x: lookup_symbols(a, SYMBOLS),
            y_: lookup_symbols(b, SYMBOLS),
        }))

1100101+1111101101 10001010010 [[[[  1.13708620e-06   6.95937388e-07  -6.49765934e-06   5.39371058e-07
     -1.72318073e-06  -3.09386269e-06  -6.02365617e-06  -4.45346586e-08
     -1.80502298e-08  -2.44404737e-06]
   [  6.59226725e-06   4.14739316e-06   3.21439006e-06   1.56650617e-06
      4.09490076e-06   4.82878795e-06   7.46406931e-06   1.79386336e-06
      5.80681353e-06  -4.94094365e-06]
   [ -5.77458934e-07  -1.84843975e-06   2.27852934e-06   2.74591139e-06
      2.53878466e-06   1.76992091e-06   8.97563496e-07   3.66410382e-06
     -2.67051473e-06  -3.18930688e-06]
   [ -1.42838553e-06  -5.03608419e-07   6.31467458e-07  -1.98442922e-06
     -1.86154477e-06  -4.33218838e-06   1.04218441e-06  -1.43082116e-06
      1.37763709e-06   2.45492174e-07]
   [ -1.51409168e-07   2.69772357e-07   1.62740804e-07  -1.49807477e-07
      1.60345735e-07   9.28652867e-07   4.12466250e-07  -9.85002316e-07
      9.81163168e-08   1.04851431e-07]]

  [[ -8.26410314e-07  -4.71084786e-06  -7.39826191e-